In [1]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import json

In [2]:
# Load the small generative model for Q&A
flan_t5_qa = pipeline("text2text-generation", model="google/flan-t5-base")


In [3]:
# Load a small, fast sentence encoder
embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [4]:

with open("summary_db.json", "r", encoding="utf-8") as f:
    summary_db = json.load(f)

print(f"✅ Loaded {len(summary_db)} articles from summary DB.")


✅ Loaded 4 articles from summary DB.


In [5]:
def answer_from_article_flan_t5(question, summary, full_text):
    if not summary and not full_text:
        return "(No content available.)", 0.0

    context_parts = []
    if summary:
        context_parts.append(summary)
    if full_text:
        context_parts.append(full_text)

    context = "\n\n".join(context_parts)

    if len(context.split()) > 800:
        context = " ".join(context.split()[:800])

    # ✅ Use better, instruct-style prompt
    prompt = f"""You are an expert news analyst.
    Based on the context below, provide a detailed and factual answer to the question.

### Question:
{question}

### Context:
{context}

### Answer:"""

    try:
        response = flan_t5_qa(prompt,
                    max_new_tokens=300,  # allow more tokens
                    min_new_tokens=80,  # encourage longer output
                    do_sample=False,
                    early_stopping=False
                    )[0]["generated_text"].strip()

        return response, 1.0
    except Exception as e:
        print(f"❌ Flan-T5 QA error: {e}")
        return "(Error while answering)", 0.0



In [6]:
def get_relevant_articles(question, summary_db, top_k=5):
    article_texts = []
    article_keys = []

    for title, entry in summary_db.items():
        summary = entry.get("summary", "")
        if summary:
            article_keys.append(title)
            article_texts.append(f"{title}. {summary}")
    
    # Embed question and articles
    question_embedding = embedder.encode(question, convert_to_tensor=True)
    article_embeddings = embedder.encode(article_texts, convert_to_tensor=True)

    # Compute cosine similarity
    similarities = util.pytorch_cos_sim(question_embedding, article_embeddings)[0]
    top_results = similarities.topk(top_k)

    # Get top titles
    top_titles = [article_keys[i] for i in top_results[1]]
    return top_titles


In [7]:
# import re

# def extractive_fallback(question, summary, full_text):
#     def best_match(sentence_pool):
#         question_words = set(re.findall(r'\b\w+\b', question.lower()))
#         best_sent, max_hits = "", 0
#         for sent in sentence_pool:
#             sent_words = set(re.findall(r'\b\w+\b', sent.lower()))
#             hits = len(question_words & sent_words)
#             if hits > max_hits:
#                 max_hits, best_sent = hits, sent
#         return best_sent if max_hits > 0 else None

#     fallback_sources = []
#     if summary:
#         summary_sentences = re.split(r'(?<=[.!?])\s+', summary.strip())
#         summary_hit = best_match(summary_sentences)
#         if summary_hit:
#             fallback_sources.append(f"Summary fallback: {summary_hit}")

#     if full_text:
#         full_sentences = re.split(r'(?<=[.!?])\s+', full_text.strip())
#         text_hit = best_match(full_sentences)
#         if text_hit:
#             fallback_sources.append(f"Full text fallback: {text_hit}")

#     return "\n".join(fallback_sources) if fallback_sources else "(Fallback found nothing meaningful.)"

In [8]:
def answer_question_over_db_flan_t5(question, summary_db, max_results=1):
    top_titles = get_relevant_articles(question, summary_db, top_k=max_results)
    top_answers = []

    for title in top_titles:
        entry = summary_db[title]
        summary = entry.get("summary", "")
        full_text = entry.get("full_text", "")
        url = entry.get("url", "")

        answer, _ = answer_from_article_flan_t5(question, summary, full_text)

        top_answers.append({
            "title": title,
            "url": url,
            "answer": answer
        })

    return top_answers


In [9]:
test_question = "What allows surgeons to see through blood?"
answers = answer_question_over_db_flan_t5(test_question, summary_db)
for ans in answers:
    print(f"Title: {ans['title']}\nAnswer: {ans['answer']}\nURL: {ans['url']}\n")


Token indices sequence length is longer than the specified maximum sequence length for this model (932 > 512). Running this sequence through the model will result in indexing errors


Title: This Experimental Tech Allows Surgeons to See Through Blood
Answer: HemoLucence. It reportedly uses AI-powered physics to digitally visualize blood as though it were translucent, which should give surgeons a clear view of the tissue beneath while operating. The technology is part of a surgical microscope system that the company plans to test in clinical trials as early as this year. The technology is part of a surgical microscope system that the company plans to test in clinical trials as early as this year.
URL: https://gizmodo.com/this-experimental-tech-allows-surgeons-to-see-through-blood-2000620162

